In [1]:
import os
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
from torchvision import datasets, transforms

In [2]:
import sys

COLAB = 'google.colab' in str(get_ipython())
TRANSFORMS = True
HAS_GPU = torch.cuda.is_available()

if COLAB: # running on Colab
    
    from google.colab import drive
    drive.mount('/content/drive')
    gdrive_path = '/content/drive//MyDrive/DL4CV-2022/project-I/'
    sys.path.append(gdrive_path)

    zip_path_train = gdrive_path + 'data/Final_Training.zip'
    zip_path_val = gdrive_path + 'data/Final_Validation.zip'
    !unzip -q "{zip_path_train}"
    !unzip -q "{zip_path_val}"
    
    if HAS_GPU:
        print("Using GPU")
else:
    path = 'data'
    HAS_GPU = False
    print("Using CPU")
    
import utils.helpers as utils
import loader.gtsrb_data as dataset

# If we run cuda then accomodate these datatype.
FloatTensor = torch.cuda.FloatTensor if HAS_GPU else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if HAS_GPU else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if HAS_GPU else torch.ByteTensor
Tensor = FloatTensor

Using CPU


In [ ]:
if COLAB:
  !ls /content/Final_Training/Images
  !ls /content/Final_Validation

In [3]:
parser = argparse.ArgumentParser()

# Load and train arguments
parser.add_argument('--data_dir_train', type=str, default='/content/Final_Training/Images',
                   help='data directory containing training image class folders')
parser.add_argument('--data_dir_test', type=str, default='data/Final_Test',
                   help='data directory containing test images')
parser.add_argument('--annotations_file', type=str, default='/content/Final_Training/Annotations',
                   help='data directory containing class annotations file')
parser.add_argument('--data_dir_val', type=str, default='/content/Final_Validation',
                   help='data directory containing validation images')
parser.add_argument('--log_dir', type=str, default='logs',
                   help='directory containing logs')
parser.add_argument('--save_dir', type=str, default='save',
                   help='directory to store checkpointed models')
parser.add_argument('--batch_size', type=int, default=50,
                   help='input batch size for training')
parser.add_argument('--num_epochs', type=int, default=5,
                   help='number of epochs to train')
parser.add_argument('--learning_rate', type=float, default=0.0001,
                    help='learning rate (default: 0.0001)')
parser.add_argument('--seed', type=int, default=1,
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=100,
                    help='how many batches to wait before logging training status')
parser.add_argument('--save_every', type=int, default=1000,
                   help='save frequency')
parser.add_argument('--decay_rate', type=float, default=0.97,
                   help='decay rate for rmsprop')
parser.add_argument('--gpu_mem', type=float, default=0.666,
                   help='%% of gpu memory to be allocated to this process. Default is 66.6%%')
parser.add_argument('--init_from', type=str, default=None,
                   help="""continue training from saved model at this path. Path must contain files saved by previous training process:
                        'config.pkl'        : configuration;
                        'words_vocab.pkl'   : vocabulary definitions;
                        'checkpoint'        : paths to model file(s) (created by tf).
                                              Note: this file contains absolute paths, be careful when moving files around;
                        'model.ckpt-*'      : file(s) with model definition (created by tf)
                    """)
args = parser.parse_args(args=[])
print(args)

Namespace(annotations_file='data/Final_Training/Annotations', batch_size=50, data_dir_test='data/Final_Test', data_dir_train='data/Final_Training/Images', data_dir_val='data/Final_Validation', decay_rate=0.97, gpu_mem=0.666, init_from=None, learning_rate=0.0001, log_dir='logs', log_interval=100, num_epochs=100, save_dir='save', save_every=1000, seed=1)


In [4]:
#transform=TRANSFORM_IMG

if COLAB:
    TRAIN_DATA_PATH = args.data_dir_train
    VAL_DATA_PATH = args.data_dir_val
    TEST_DATA_PATH = gdrive_path+args.data_dir_test
    print(args.data_dir_train)
    print(args.data_dir_val)

else:
    TRAIN_DATA_PATH = os.getcwd()+'/'+args.data_dir_train
    VAL_DATA_PATH = os.getcwd()+'/'+args.data_dir_val
    TEST_DATA_PATH = os.getcwd()+'/'+args.data_dir_test
    
BATCH_SIZE = args.batch_size
EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
LEARNING_RATE = args.learning_rate

In [5]:
from loader import transforms as tfs

if TRANSFORMS:

    # Apply data transformation to augment training imageset.
    train_loader = torch.utils.data.DataLoader(datasets.ImageFolder(
                                                TRAIN_DATA_PATH,
                                                transform=tfs.data_transforms), 
                                                batch_size=args.batch_size, 
                                                shuffle=True, num_workers=2, 
                                                pin_memory=HAS_GPU)
    
    val_loader = torch.utils.data.DataLoader(datasets.ImageFolder(
                                                VAL_DATA_PATH,
                                                transform=tfs.data_transforms),
                                                batch_size=args.batch_size, 
                                                shuffle=False, num_workers=2, 
                                                pin_memory=HAS_GPU)
    
    # Apply data transformations to augment the training imageset.
   # train_loader = torch.utils.data.DataLoader(
   #     torch.utils.data.ConcatDataset([torchvision.datasets.ImageFolder(TRAIN_DATA_PATH,
   #     transform=transforms.data_transforms),torchvision.datasets.ImageFolder(TRAIN_DATA_PATH,
   #     transform=transforms.data_jitter_contrast),torchvision.datasets.ImageFolder(TRAIN_DATA_PATH,
   #     transform=transforms.data_grayscale),torchvision.datasets.ImageFolder(TRAIN_DATA_PATH,
   #     transform=transforms.data_translate)]), 
   #         batch_size=args.batch_size, 
   #         shuffle=True, num_workers=2, 
   #         pin_memory=HAS_GPU)

else: # In this case train=39209, test=12630, val=3870
    train_data        = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, 
                                                         transform=tfs.data_transforms)
    train_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2)
    test_data         = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, 
                                                         transform=tfs.data_transforms)
    test_loader  = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_data          = torchvision.datasets.ImageFolder(root=VAL_DATA_PATH, 
                                                         transform=tfs.data_transforms)
    val_loader   = data.DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2)
    
# Order: data_transforms, data_jitter_contrast, data_grayscale, data_translate


In [6]:
if COLAB:
  print(len(train_loader))
  print(len(val_loader))
else:
  print(len(train_data))
  print(len(test_data))
  print(len(val_data))

39209
12630
3870


In [7]:
# Neural Network and Optimizer
from model.models import Net as NN
model = NN()

if HAS_GPU:
    model.cuda()
    
optimizer = optim.Adam(filter(lambda p: p.requires_grad,
                              model.parameters()),
                              lr=args.learning_rate)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 'min',
                                                 patience=5,
                                                 factor=0.5,
                                                 verbose=True)

In [9]:
def train(epoch):
    model.train()
    correct = 0
    training_loss = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        if HAS_GPU:
            data = data.cuda()
            target = target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        max_index = output.max(dim = 1)[1]
        correct += (max_index == target).sum()
        training_loss += loss
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss per example: {:.6f}\tLoss: {:.6f}'.format(
                        epoch, batch_idx * len(data), len(train_loader.dataset),
                        100. * batch_idx / len(train_loader), loss.data.item()/(args.batch_size * args.log_interval),
                        loss.data.item()))
            print('\nTraining set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                training_loss / len(train_loader.dataset), correct, len(train_loader.dataset),
                100. * correct / len(train_loader.dataset)))
    

In [10]:
def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        with torch.no_grad():
            data, target = Variable(data), Variable(target)
            if HAS_GPU:
                data = data.cuda()
                target = target.cuda()
            output = model(data)
            validation_loss += F.nll_loss(output, target, size_average=False).data.item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    scheduler.step(np.around(validation_loss,2))
    
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
    

In [ ]:
for epoch in range(1, args.num_epochs + 1):
    train(epoch)
    validation()

Train Epoch: 1 [0/39209 (0%)]	Loss per example: 0.000756	Loss: 3.780284

Training set: Average loss: 0.0001, Accuracy: 0/39209 (0%)

Train Epoch: 1 [5000/39209 (13%)]	Loss per example: 0.000663	Loss: 3.313797

Training set: Average loss: 0.0092, Accuracy: 404/39209 (1%)

Train Epoch: 1 [10000/39209 (25%)]	Loss per example: 0.000604	Loss: 3.020405

Training set: Average loss: 0.0173, Accuracy: 1273/39209 (3%)

Train Epoch: 1 [15000/39209 (38%)]	Loss per example: 0.000491	Loss: 2.455574

Training set: Average loss: 0.0245, Accuracy: 2564/39209 (7%)

Train Epoch: 1 [20000/39209 (51%)]	Loss per example: 0.000493	Loss: 2.463892

Training set: Average loss: 0.0309, Accuracy: 4181/39209 (11%)

Train Epoch: 1 [25000/39209 (64%)]	Loss per example: 0.000425	Loss: 2.123121

Training set: Average loss: 0.0364, Accuracy: 6198/39209 (16%)

Train Epoch: 1 [30000/39209 (76%)]	Loss per example: 0.000320	Loss: 1.597590

Training set: Average loss: 0.0412, Accuracy: 8549/39209 (22%)

Train Epoch: 1 [3500

/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_m1/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: dlopen(/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_m1/lib/python3.8/site-packages/torchvision/image.so, 0x0006): symbol not found in flat namespace '__ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE'
  warn(f"Failed to load image Python extension: {e}")
/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_m1/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: dlopen(/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_m1/lib/python3.8/site-packages/torchvision/image.so, 0x0006): symbol not found in flat namespace '__ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_1


Validation set: Average loss: 1.6887, Accuracy: 2130/3870 (55%)

Train Epoch: 2 [0/39209 (0%)]	Loss per example: 0.000267	Loss: 1.336278

Training set: Average loss: 0.0000, Accuracy: 29/39209 (0%)

Train Epoch: 2 [5000/39209 (13%)]	Loss per example: 0.000258	Loss: 1.288401

Training set: Average loss: 0.0031, Accuracy: 3326/39209 (8%)

Train Epoch: 2 [10000/39209 (25%)]	Loss per example: 0.000204	Loss: 1.020322

Training set: Average loss: 0.0057, Accuracy: 6891/39209 (18%)

Train Epoch: 2 [15000/39209 (38%)]	Loss per example: 0.000167	Loss: 0.837024

Training set: Average loss: 0.0079, Accuracy: 10701/39209 (27%)

Train Epoch: 2 [20000/39209 (51%)]	Loss per example: 0.000137	Loss: 0.683697

Training set: Average loss: 0.0097, Accuracy: 14700/39209 (37%)

Train Epoch: 2 [25000/39209 (64%)]	Loss per example: 0.000095	Loss: 0.474875

Training set: Average loss: 0.0113, Accuracy: 18908/39209 (48%)

Train Epoch: 2 [30000/39209 (76%)]	Loss per example: 0.000117	Loss: 0.586917

Training se

/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_m1/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: dlopen(/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_m1/lib/python3.8/site-packages/torchvision/image.so, 0x0006): symbol not found in flat namespace '__ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE'
  warn(f"Failed to load image Python extension: {e}")
/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_m1/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: dlopen(/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_m1/lib/python3.8/site-packages/torchvision/image.so, 0x0006): symbol not found in flat namespace '__ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_1


Validation set: Average loss: 0.2956, Accuracy: 3617/3870 (93%)

Train Epoch: 3 [0/39209 (0%)]	Loss per example: 0.000080	Loss: 0.398937

Training set: Average loss: 0.0000, Accuracy: 44/39209 (0%)

Train Epoch: 3 [5000/39209 (13%)]	Loss per example: 0.000056	Loss: 0.280790

Training set: Average loss: 0.0010, Accuracy: 4546/39209 (12%)

Train Epoch: 3 [10000/39209 (25%)]	Loss per example: 0.000085	Loss: 0.425730

Training set: Average loss: 0.0019, Accuracy: 9095/39209 (23%)

Train Epoch: 3 [15000/39209 (38%)]	Loss per example: 0.000034	Loss: 0.170600

Training set: Average loss: 0.0027, Accuracy: 13684/39209 (35%)

Train Epoch: 3 [20000/39209 (51%)]	Loss per example: 0.000065	Loss: 0.326387

Training set: Average loss: 0.0035, Accuracy: 18320/39209 (47%)

Train Epoch: 3 [25000/39209 (64%)]	Loss per example: 0.000037	Loss: 0.185411

Training set: Average loss: 0.0041, Accuracy: 22992/39209 (59%)

Train Epoch: 3 [30000/39209 (76%)]	Loss per example: 0.000062	Loss: 0.307867

Training s